In [31]:
import pandas as pd
import numpy as np
import utils
import h5py
import basic_adapter_utils
import torch
import whisperx

In [35]:
'''
tiny, 
base
small
medium, 
large-v1, large-v2, large-v3, large
'''

'\ntiny, \nbase\nsmall\nmedium, \nlarge-v1, large-v2, large-v3, large\n'

In [33]:
device = "cuda"

pure_model = whisperx.load_model('medium', download_root='models/', language='de', compute_type='float16', device=device)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../home/dpandya/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.1+cu121. Bad things might happen unless you revert torch to 1.x.


In [23]:
import unicodedata
import re

def normalize_string(s):
    # Convert to lower case
    s = s.lower()
    
    # Remove punctuation
    s = re.sub(r'[^\w\s]', '', s)
    
    # Additional replacements
    s = s.replace('ß', 'ss')
    s = s.replace('ö', 'oe')
    s = s.replace('ä', 'ae')
    s = s.replace('ü', 'ue')
    
    # Convert umlauts and special characters to normal English characters
    s = unicodedata.normalize('NFD', s).encode('ascii', 'ignore').decode('utf-8')
    
    return s

In [24]:
df = pd.read_csv('LibriVox_Kaggle_officeRIR_whisperx.csv')

In [25]:
df['orgText'] = df['orgText'].apply(normalize_string)
df['whisperxOut'] = df['whisperxOut'].apply(normalize_string)
df['rir_whisperxOut'] = df['rir_whisperxOut'].apply(normalize_string)

In [27]:
import jiwer

df['whisperxOut_wer'] = df.apply(lambda row: jiwer.wer(row['orgText'], row['whisperxOut']), axis=1)
df['rir_whisperxOut_wer'] = df.apply(lambda row: jiwer.wer(row['orgText'], row['rir_whisperxOut']), axis=1)

In [29]:
df['whisperxOut_wer'].mean(), df['rir_whisperxOut_wer'].mean()

(0.06404120085619888, 0.07375931517350354)

In [39]:
df = df.drop(['WER'], axis=1)

In [41]:
df.to_csv('LibriVox_Kaggle_officeRIR_whisperx.csv', index=False)